In [1]:
import random
import numpy as np

In [2]:


class ACO:
    def __init__(self, num_ants, num_iterations, decay, alpha, beta, graph):
        self.num_ants = num_ants
        self.num_iterations = num_iterations
        self.decay = decay
        self.alpha = alpha  # influence of pheromone on direction
        self.beta = beta    # influence of heuristic value (distance)
        self.graph = graph  # adjacency matrix of distances
        self.pheromone = np.ones(self.graph.shape) / len(graph)

    def run(self):
        shortest_path = None
        best_cost = float('inf')
        for _ in range(self.num_iterations):
            all_paths = self.construct_paths()
            self.update_pheromone(all_paths)
            shortest_path, best_cost = self.find_best_path(all_paths, best_cost, shortest_path)
        return shortest_path, best_cost

    def construct_paths(self):
        all_paths = []
        for _ in range(self.num_ants):
            path = self.construct_path(0)  # assuming all ants start at node 0
            all_paths.append((path, self.path_cost(path)))
        return all_paths

    def construct_path(self, start):
        path = []
        visited = set()
        current = start
        path.append(current)
        visited.add(current)
        while len(visited) < len(self.graph):
            next_node = self.select_next_node(current, visited)
            path.append(next_node)
            visited.add(next_node)
            current = next_node
        return path

    def select_next_node(self, current, visited):
        probabilities = self.calculate_probabilities(current, visited)
        next_node = random.choices(range(len(self.graph)), weights=probabilities)[0]
        return next_node

    def calculate_probabilities(self, current, visited):
        pheromone = np.power(self.pheromone[current], self.alpha)
        heuristic = np.power(1.0 / self.graph[current], self.beta)  # assuming graph contains distances
        probabilities = pheromone * heuristic
        probabilities[list(visited)] = 0  # set probabilities of visited nodes to 0
        probabilities /= probabilities.sum()
        return probabilities

    def update_pheromone(self, all_paths):
        self.pheromone *= (1 - self.decay)  # evaporation
        for path, cost in all_paths:
            for step in range(len(path) - 1):
                self.pheromone[path[step]][path[step+1]] += 1 / cost

    def find_best_path(self, all_paths, best_cost, shortest_path):
        for path, cost in all_paths:
            if cost < best_cost:
                shortest_path = path
                best_cost = cost
        return shortest_path, best_cost

    def path_cost(self, path):
        cost = 0
        for i in range(len(path) - 1):
            cost += self.graph[path[i]][path[i+1]]
        return cost


In [3]:
import numpy as np

# Define the adjacency matrix for a 5-node graph
# Using a symmetric matrix with diagonal zeros (no self-loop) and random distances
np.random.seed(42)  # for reproducible results
num_nodes = 5
graph = np.random.rand(num_nodes, num_nodes)
graph = (graph + graph.T) / 2  # Make the matrix symmetric
np.fill_diagonal(graph, 0)  # No self-loops

# Example of the graph matrix
print("Graph (Adjacency Matrix):")
print(graph)

# Set up the ACO parameters
num_ants = 5
num_iterations = 100
decay = 0.1
alpha = 1.0  # Influence of pheromone
beta = 1.0   # Influence of heuristic (inverse of distance here)

# Initialize ACO
aco = ACO(num_ants, num_iterations, decay, alpha, beta, graph)

# Run ACO
best_path, best_cost = aco.run()

# Print the results
print("\nBest Path Found:", best_path)
print("Cost of the Best Path:", best_cost)


Graph (Adjacency Matrix):
[[0.         0.55335441 0.37628922 0.3910315  0.38393577]
 [0.55335441 0.         0.918043   0.45267863 0.42378322]
 [0.37628922 0.918043   0.         0.36854777 0.23698481]
 [0.3910315  0.45267863 0.36854777 0.         0.32879549]
 [0.38393577 0.42378322 0.23698481 0.32879549 0.        ]]


C:\Users\rene_\AppData\Local\Temp\ipykernel_7216\149946257.py:47: RuntimeWarning: divide by zero encountered in divide
  heuristic = np.power(1.0 / self.graph[current], self.beta)  # assuming graph contains distances



Best Path Found: [0, 3, 2, 4, 1]
Cost of the Best Path: 1.4203472952756953
